<h2>Autocomplete for StackOverflow</h2>

In [46]:
import os
import numpy as np
from keras.preprocessing.text import Tokenizer 
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras import layers as L
import keras.utils as ku 
import random
import pandas as pd
from keras.models import load_model
import pickle

In [8]:
os.chdir("/content/drive/MyDrive/Python/Practice/Auto Complete stack overflow")

In [9]:
ls

'Autocomplete '   tokenizer.pickle   train.csv   valid.csv   weights/


In [10]:
df = pd.read_csv("train.csv")

In [11]:
df.head()

,Id,Title,Body,Tags,CreationDate,Y
0,34552656,Java: Repeat Task Every Random Seconds,<p>I'm already familiar with repeating tasks e...,<java><repeat>,2016-01-01 00:21:59,LQ_CLOSE
1,34553034,Why are Java Optionals immutable?,<p>I'd like to understand why Java 8 Optionals...,<java><optional>,2016-01-01 02:03:20,HQ
2,34553174,Text Overlay Image with Darkened Opacity React...,<p>I am attempting to overlay a title over an ...,<javascript><image><overlay><react-native><opa...,2016-01-01 02:48:24,HQ
3,34553318,Why ternary operator in swift is so picky?,"<p>The question is very simple, but I just cou...",<swift><operators><whitespace><ternary-operato...,2016-01-01 03:30:17,HQ
4,34553755,hide/show fab with scale animation,<p>I'm using custom floatingactionmenu. I need...,<android><material-design><floating-action-but...,2016-01-01 05:21:48,HQ


In [12]:
stack_title = df['Title'].values

In [13]:
t = Tokenizer(num_words=None,
              filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', 
              lower=True, split=' ', char_level=False, 
              oov_token=None, document_count=0)

In [ ]:
t.fit_on_texts(stack_title)

In [ ]:
total_words = len(t.word_index)+1

In [ ]:
"""
Implementing N gram 
Words will be processed like : 
Hi you
Hi you how
hi you how are

"""

def generate_seq(text):
  t.fit_on_texts(text)
  total_words = len(t.word_index)+1
  input_seq = []
  for line in text:
    token_list = t.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
      n_gram = token_list[:i+1]
      input_seq.append(n_gram)
  return input_seq , total_words

In [ ]:
input_sequences , total_words = generate_seq(stack_title)

In [ ]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len, padding = 'pre'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes = total_words)
    
    return predictors, label, max_sequence_len

In [ ]:
predictors, label, max_sequence_len = generate_padded_sequences(input_sequences)

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 20, input_length=max_sequence_len - 1))
model.add(LSTM(100))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model = create_model(max_sequence_len, total_words)
model.summary()

model.fit(predictors, label, epochs=100, verbose=5)

In [29]:
with open('tokenizer.pickle', 'rb') as handle:
    t = pickle.load(handle)

In [30]:
def autocomplete(seed_text, next_words, model, max_seq_len):
    for _ in range(next_words):
        token_list = t.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        
        predicted = model.predict_classes(token_list, verbose=0)
        
        output_word = ''
        
        for word,index in t.word_index.items():
            if index == predicted:
                output_word = word
                break
                
        seed_text = seed_text + " " + output_word
        
    return seed_text.title()

In [57]:
model = load_model('weights/model-0149.hdf5')

In [62]:
autocomplete("if else statement",random.randint(1,10),model,35)

'If Else Statement With Operator In'

In [66]:
autocomplete("i cannot",20,model,35)

'I Cannot Find The Sum Of Each Integer In The Result But I Am Getting An Array Of A Function Is Null'